In [ ]:
# https://medium.com/@sandromoreira/building-intelligent-agents-from-scratch-with-crewai-gemini-and-google-colab-e8270af54676
# crew ai agents project with google gemini-1.5-flash and duckduckgo search

In [ ]:
!pip install --q --upgrade crewai
!pip install --q --upgrade crewai-tools
!pip install --q --upgrade langchain-google-genai
!pip install --q --upgrade duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from crewai import Agent, Crew, Process, Task, LLM
from crewai_tools import tool

In [ ]:
from google.colab import userdata
geminikey = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = geminikey

In [ ]:
my_llm = LLM(
    model='gemini/gemini-1.5-flash',
    api_key=geminikey
)

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

@tool('DuckDuckGoSearch')
def search_tool(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

In [ ]:
searcher = Agent(
    role = 'Content Searcher',
    goal = 'Search for online content about {topic}',
    backstory = '''
                You will be working on creating articles for LinkedIn about {topic}.
                You will conduct research online, collect and group information.
                Your work will serve as a basis for the Content Writer.
                ''',
    llm = my_llm,
    verbose=True,
    allow_delegate=False,
    tools = [search_tool]
)

writer = Agent(
    role = 'Content Writer',
    goal = 'Writing a LinkedIn post about {topic}',
    backstory = ''''
                Your task is to create a LinkedIn article on {topic}, using the data gathered by the Content Searcher as a reference.
                The article should be engaging and informative, maintaining factual accuracy
                ''',
    tools = [search_tool],
    llm=my_llm,
    verbose=True
)

editor = Agent(
    role = 'Content Editor',
    goal = 'Revise the LinkedIn post to guarantee factual accuracy while maintaining a relaxed and entertaining style.',
    backstory = ''''
                Your task is to review and edit a LinkedIn article written by the Content Writer.
                You will ensure that the final version, which should be accurate and well-written.
                ''',
    tools = [search_tool],
    llm=my_llm,
    verbose=True
)

In [ ]:
search = Task(
    description=
        '''
         1. Give priority to the most recent trends, key figures, and breaking news related to {topic}.
         2. Determine the target audience, taking into account their interests and challenges.
         3. Incorporate SEO keywords and data from credible sources.
        ''',
    agent=searcher,
    expected_output='A comprehensive trend plan focused on {topic}, incorporating top SEO keywords and the most recent news.'
)


write = Task(
    description='''
        1. Leverage the gathered content data to develop a compelling LinkedIn post centered around the theme of {topic}.
        2. Seamlessly integrate relevant SEO keywords into the content.
        3. Structure the post to be captivating, with a conclusion designed to stimulate reflection.
        ''',
    agent=writer,
    expected_output='A LinkedIn post about {topic}.'
)

review = Task(
    description=("Review the LinkedIn post in question for grammatical errors."
    ),
    agent=editor,
    expected_output='A LinkedIn post ready for publication. The text is formatted in paragraphs and does not use bullet points.'
)

In [ ]:
crew = Crew(
    agents=[searcher,writer,editor],
    tasks=[search,write,review],
    verbose=1,
    process=Process.sequential
)

In [ ]:
subject = 'Career opportunities for developers specializing in Multi-Agent Systems'
template_input = {"topic": subject}
result = crew.kickoff(inputs=template_input)

# Agent: Content Searcher
## Task: 
         1. Give priority to the most recent trends, key figures, and breaking news related to Career opportunities for developers specializing in Multi-Agent Systems.
         2. Determine the target audience, taking into account their interests and challenges.
         3. Incorporate SEO keywords and data from credible sources.
        


/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(




# Agent: Content Searcher
## Thought: Thought: I need to find information on career opportunities for developers specializing in Multi-Agent Systems (MAS), focusing on recent trends, key figures, and news.  I also need to identify the target audience and relevant SEO keywords.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"career opportunities multi agent systems developers\"}"
## Tool Output: 
268 Multi Agent Systems Jobs in United States · Shift Manager (Payne) $5000 Sign-on Bonus. Shift Manager (Payne) $5000 Sign-on Bonus. · Sales Lead. Sales Lead. software engineering for multi agent systems jobs · VP of AI Research, Trilogy (Remote) - $200,000/year USD. New · Electrical Engineer. GreenXT · Controls Engineer. Group Air Agent · ADTRAV Travel Management Boise, ID ; Software Developer · Nagios Enterprises, LLC Saint Paul, MN ; Customer Service Agent (Hybrid). Contact Center ... Honda Research Institute USA (HRI-US) is looking for a senior research engineer wit

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(




# Agent: Content Searcher
## Thought: Thought: The search results show various job postings, but lack information on trends, key figures, and a comprehensive overview of the career landscape. I need to refine my search.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"multi agent systems trends career\"}"
## Tool Output: 
Current Trends and Innovations in Multi-Agent Systems. The world of artificial intelligence is rapidly evolving, and multi-agent systems (MAS) are at the forefront of this transformation. These sophisticated networks of AI entities are being revolutionized by two key technologies: generative AI and AutoML (Automated Machine Learning). Trend #1: Why Multi-Agent Systems (MAS) Are the Next Big Thing . What It Is: Multi-Agent Systems (MAS) involve multiple AI programs, or "agents," working together to achieve specific goals, often in real-time and with the ability to be "sensitive" to changes in the environment. This doesn't mean that agentic AI is 

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(




# Agent: Content Searcher
## Thought: Thought: This provides some trend information, but I still need more specific data on the job market and target audience.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"multi agent systems job market outlook\"}"
## Tool Output: 
AI agents — LLM-based bots that can independently assign themselves to and complete tasks to fulfill a user's goal — took off like wildfire in 2023.. Now, multi-agent AI systems are emerging as the next iteration to provide a more effective approach to executing complex tasks. Multi-agent systems string together agents that "talk" to each other and collaborate. In finance, for instance, multi-agent systems are transforming market analysis and risk assessment. These systems can process vast amounts of data from diverse sources, providing comprehensive insights that would be impossible for a single AI model or human analyst to achieve. ... Multi-agent systems offer unparalleled scalability, allowing .

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(




# Agent: Content Editor
## Thought: Thought:The LinkedIn post is well-written but could benefit from improved flow and some minor grammatical corrections.  The bullet points should be incorporated into paragraphs for better readability on LinkedIn.  I need to check the factual accuracy of the claims about MAS applications in various industries.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"applications of multi-agent systems in finance, healthcare, robotics, and logistics\"}"
## Tool Output: 
Multi-agent systems in healthcare utilize the advantages of both sub-fields. Different diseases can be addressed through task division as per the norms of MAS.M. Bhanu Sridhar, researcher in healthcare applications of MAS. Steering Transportation into the Future. Multi-agent systems are paving the way for smarter, more efficient transportation ... Multi-agent systems (MAS) are transforming industrial automation by enabling more flexible, adaptive, and efficient manufactur